In [1]:
import os
import shutil
import random

# ソースフォルダとターゲットフォルダのパスを設定
source_folder = "/Users/iniad/Downloads/dataset_preprocessing_for_vulnerabilities/reentrancy/sourcecode/ast3"
target_folder = "/Users/iniad/Downloads/dataset_preprocessing_for_vulnerabilities/testcode/reentrancy"

# ソースフォルダ内のファイルリストを取得
file_list = os.listdir(source_folder)

# ランダムにファイルを選ぶ (例: 10個のファイルを選ぶ)
num_files_to_select = 10
selected_files = random.sample(file_list, num_files_to_select)

# 選択されたファイルをターゲットフォルダにコピー
for file_name in selected_files:
    source_file_path = os.path.join(source_folder, file_name)
    target_file_path = os.path.join(target_folder, file_name)
    shutil.copy(source_file_path, target_file_path)

print(f"{num_files_to_select} files copied to {target_folder}")


10 files copied to /Users/iniad/Downloads/dataset_preprocessing_for_vulnerabilities/testcode/reentrancy


In [55]:
import os
import shutil
import random

# ソースフォルダとターゲットフォルダのパスを設定
source_folder = "/Users/iniad/Downloads/dataset_preprocessing_for_vulnerabilities/contract_dataset_ethereum/contract27"
target_folder = "/Users/iniad/Downloads/dataset_preprocessing_for_vulnerabilities/testcode/real"

# ソースフォルダ内のファイルリストを取得
file_list = os.listdir(source_folder)

# ランダムにファイルを選ぶ (例: 10個のファイルを選ぶ)
num_files_to_select = 5
selected_files = random.sample(file_list, num_files_to_select)

# 選択されたファイルをターゲットフォルダにコピー
for file_name in selected_files:
    source_file_path = os.path.join(source_folder, file_name)
    target_file_path = os.path.join(target_folder, file_name)
    shutil.copy(source_file_path, target_file_path)

print(f"{num_files_to_select} files copied to {target_folder}")


5 files copied to /Users/iniad/Downloads/dataset_preprocessing_for_vulnerabilities/testcode/real


In [51]:
import os
import random
import shutil

def filter_files_by_content(source_folder, target_folder, num_files, content_keywords, exclusion_keywords=None):
    if not os.path.exists(source_folder) or not os.path.exists(target_folder):
        print("Source or target folder does not exist.")
        return

    selected_files = []
    for root, _, files in os.walk(source_folder):
        for file in files:
            file_path = os.path.join(root, file)
            with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
                content = f.read()
                contains_all_keywords = all(keyword in content for keyword in content_keywords)
                if contains_all_keywords and (exclusion_keywords is None or not any(keyword in content for keyword in exclusion_keywords)):
                    selected_files.append(file_path)

    if len(selected_files) < num_files:
        print("Not enough files matching the criteria.")
        return

    random.shuffle(selected_files)
    for file_path in selected_files[:num_files]:
        file_name = os.path.basename(file_path)
        target_path = os.path.join(target_folder, file_name)
        shutil.copy(file_path, target_path)
        #print(f"File '{file_name}' copied to '{target_folder}'")

source_folder = "/Users/iniad/Downloads/dataset_preprocessing_for_vulnerabilities/contract_dataset_ethereum/contract27"
target_folder = "/Users/iniad/Downloads/dataset_preprocessing_for_vulnerabilities/testcode/real"

num_files_to_copy = 10
content_keywords = ["call.value"]
exclusion_keywords = ["noreentrancy"]

filter_files_by_content(source_folder, target_folder, num_files_to_copy, content_keywords, exclusion_keywords)
#print(f"{num_files_to_copy} files copied to {target_folder}")

10 files copied to /Users/iniad/Downloads/dataset_preprocessing_for_vulnerabilities/testcode/real


1. call.valueの探索

In [78]:
#reentrancyのcall.value
import subprocess
import os
import json

contract_directory = "/Users/iniad/Downloads/dataset_preprocessing_for_vulnerabilities/testcode/reentrancy"
solidity_files = os.listdir(contract_directory)
solidity_filesX1 = []
def find_call_value(node):
    if isinstance(node, dict):
        # ノードが "MemberAccess" で "memberName" が "value" の場合
        if node.get("nodeType") == "MemberAccess" and node.get("memberName") == "value" and "expression" in node:
            if "memberName" in node["expression"] and node["expression"]["memberName"] == "call":
                return node

        # ノードが辞書の場合、再帰的に探索
        for key, value in node.items():
            result = find_call_value(value)
            if result:
                return result
    elif isinstance(node, list):
        # ノードがリストの場合、リスト内の各要素を再帰的に探索
        for item in node:
            result = find_call_value(item)
            if result:
                return result

# SolidityのASTを代入する変数（ここではast_dataと仮定）
res = 0
for solidity_file in solidity_files:
    file_path = os.path.join(contract_directory, solidity_file)
    with open(file_path, "r") as file:
        ast_data = json.load(file)
        if ast_data:
            # ast_data内で"call.value"を探す
            found_node = find_call_value(ast_data)

            # "call.value"が見つかった場合
            if found_node:
                #print(f"Found 'call.value' in Solidity code: {solidity_file}")
                res += 1
                solidity_filesX1.append(solidity_file)
    

print(f"Total 'call.value' found in {res} files.")
print(f"files: {solidity_filesX1}")

Total 'call.value' found in 10 files.
files: ['ast_output_40754.sol.json', 'ast_output_23792.sol.json', 'ast_output_30046.sol.json', 'ast_output_35661.sol.json', 'ast_output_40737.sol.json', 'ast_output_50013.sol.json', 'ast_output_40780.sol.json', 'ast_output_26188.sol.json', 'ast_output_50003.sol.json', 'ast_output_40746.sol.json']


In [80]:
#real_astのcall.value
import subprocess
import os
import json

contract_directory = "/Users/iniad/Downloads/dataset_preprocessing_for_vulnerabilities/testcode/real_ast"
solidity_files = os.listdir(contract_directory)
solidity_filesY1 = []
def find_call_value(node):
    if isinstance(node, dict):
        # ノードが "MemberAccess" で "memberName" が "value" の場合
        if node.get("nodeType") == "MemberAccess" and node.get("memberName") == "value" and "expression" in node:
            if "memberName" in node["expression"] and node["expression"]["memberName"] == "call":
                return node

        # ノードが辞書の場合、再帰的に探索
        for key, value in node.items():
            result = find_call_value(value)
            if result:
                return result
    elif isinstance(node, list):
        # ノードがリストの場合、リスト内の各要素を再帰的に探索
        for item in node:
            result = find_call_value(item)
            if result:
                return result

# SolidityのASTを代入する変数（ここではast_dataと仮定）
res = 0
r1 = 0
for solidity_file in solidity_files:
    file_path = os.path.join(contract_directory, solidity_file)
    with open(file_path, "rb") as file:
        file_data = file.read()
        try:
            ast_data = file_data.decode("utf-8")
            ast_data = json.loads(ast_data)
            # ファイルデータをUTF-8でデコードしてASTに変換
            if ast_data:
                # ast_data内で指定の修飾子を探す
                found_node = find_call_value(ast_data)

                # "call.value"が見つかった場合
                if found_node:
                    res += 1
                    solidity_filesY1.append(solidity_file)
                    print(f"Total 'call.value' found in :{solidity_file}")
                    
        except UnicodeDecodeError:
            # ファイルデータがUTF-8でデコードできない場合
            r1 += 1
            #print(f"File contains non-UTF-8 encoded data {solidity_file}")
print(f"Total 'call.value' found in {res} files.")
print(f"files: {solidity_filesY1}")

Total 'call.value' found in :ast_output_26632.sol.json
Total 'call.value' found in :ast_output_26250.sol.json
Total 'call.value' found in :ast_output_26948.sol.json
Total 'call.value' found in :ast_output_26935.sol.json
Total 'call.value' found in :ast_output_26600.sol.json
Total 'call.value' found in 5 files.
files: ['ast_output_26632.sol.json', 'ast_output_26250.sol.json', 'ast_output_26948.sol.json', 'ast_output_26935.sol.json', 'ast_output_26600.sol.json']


正確に結果が出た

２. 残高変更

In [119]:
#できた
import os
import json

contract_directory = "/Users/iniad/Downloads/dataset_preprocessing_for_vulnerabilities/testcode/reentrancy"
solidity_files = os.listdir(contract_directory)
solidity_filesX2 = []
res_sub = 0
#call.valueが含まれる関数
def find_call_value_in_function(node, function_name):
    if isinstance(node, dict):
        if "nodeType" in node and node["nodeType"] == "FunctionDefinition":
            function_name = node.get("name")
        if "nodeType" in node and node["nodeType"] == "MemberAccess" and node.get("memberName") == "value":
            if "expression" in node and node["expression"].get("memberName") == "call":
                youso1 = []
                parent_node = node.get("src", {})
                youso1.append(function_name)
                youso1.append(parent_node)
                find_call_value_in_function_list.append(youso1)
                #print(f"'call.value' found in function: {function_name} {parent_node}")
        for key, value in node.items():
            find_call_value_in_function(value, function_name)
    elif isinstance(node, list):
        for item in node:
            find_call_value_in_function(item, function_name)
            
#引き算が含まれる関数
def find_sub_value_in_function(node, function_name):
    if isinstance(node, dict):
        if "nodeType" in node and node["nodeType"] == "FunctionDefinition":
            function_name = node.get("name")
        if "nodeType" in node and (node["nodeType"] == "BinaryOperation" or node.get("nodeType") == "Assignment"):
            youso2 = []
            if (node["operator"] == "-" or node["operator"] == "-="):
                parent_node = node.get("src", {})
                youso2.append(function_name)
                youso2.append(parent_node)
                find_sub_value_in_function_list.append(youso2)
                #print(f"'sub' found in function: {function_name}")
            elif node.get("operator") == "=":
                right_hand_side = node.get("rightHandSide")
                if right_hand_side and right_hand_side.get("value") == "0":
                    parent_node = node.get("src", {})
                    youso2.append(function_name)
                    youso2.append(parent_node)
                    find_sub_value_in_function_list.append(youso2)
                    #print(f"'sub' found in function: {function_name}")
        for key, value in node.items():
            find_sub_value_in_function(value, function_name)
    elif isinstance(node, list):
        for item in node:
            find_sub_value_in_function(item, function_name)

for solidity_file in solidity_files:
    file_path = os.path.join(contract_directory, solidity_file)
    find_call_value_in_function_list = [] 
    find_sub_value_in_function_list = []
    with open(file_path, "rb") as file:
        file_data = file.read()
        try:
            ast_data = file_data.decode("utf-8")
            ast_data = json.loads(ast_data)
            if ast_data:
                find_call_value_in_function(ast_data, function_name=None)
                find_sub_value_in_function(ast_data, function_name=None)
                for a in find_call_value_in_function_list:
                    for b in find_sub_value_in_function_list:
                        #関数名が一緒のとき
                        if a[0]==b[0]:
                            #引き算がcall.valueよりも後にあったら
                            if a[1] < b[1]:
                                res_sub += 1
                                solidity_filesX2.append(solidity_file)
                                print(f"Found substruct after 'call.value': {solidity_file}")
        except UnicodeDecodeError:
            r1 += 1
print(f"result:{res_sub}")
print(f"files: {solidity_filesX2}")

Found substruct after 'call.value': ast_output_40754.sol.json
Found substruct after 'call.value': ast_output_50013.sol.json
Found substruct after 'call.value': ast_output_40780.sol.json
Found substruct after 'call.value': ast_output_50003.sol.json
Found substruct after 'call.value': ast_output_40746.sol.json
result:5
files: ['ast_output_40754.sol.json', 'ast_output_50013.sol.json', 'ast_output_40780.sol.json', 'ast_output_50003.sol.json', 'ast_output_40746.sol.json']


In [120]:
#できた
#call.valueが含まれる関数名
import os
import json

contract_directory = "/Users/iniad/Downloads/dataset_preprocessing_for_vulnerabilities/testcode/real_ast"
solidity_files = os.listdir(contract_directory)
solidity_filesY2 = []
res_sub = 0
def find_call_value_in_function(node, function_name):
    if isinstance(node, dict):
        if "nodeType" in node and node["nodeType"] == "FunctionDefinition":
            function_name = node.get("name")
        if "nodeType" in node and node["nodeType"] == "MemberAccess" and node.get("memberName") == "value":
            if "expression" in node and node["expression"].get("memberName") == "call":
                youso1 = []
                parent_node = node.get("src", {})
                youso1.append(function_name)
                youso1.append(parent_node)
                find_call_value_in_function_list.append(youso1)
                #print(f"'call.value' found in function: {function_name} {parent_node}")
        for key, value in node.items():
            find_call_value_in_function(value, function_name)
    elif isinstance(node, list):
        for item in node:
            find_call_value_in_function(item, function_name)

def find_sub_value_in_function(node, function_name):
    if isinstance(node, dict):
        if "nodeType" in node and node["nodeType"] == "FunctionDefinition":
            function_name = node.get("name")
        if "nodeType" in node and (node["nodeType"] == "BinaryOperation" or node.get("nodeType") == "Assignment"):
            youso2 = []
            if (node["operator"] == "-" or node["operator"] == "-="):
                parent_node = node.get("src", {})
                youso2.append(function_name)
                youso2.append(parent_node)
                find_sub_value_in_function_list.append(youso2)
                #print(f"'sub' found in function: {function_name}")
            elif node.get("operator") == "=":
                right_hand_side = node.get("rightHandSide")
                if right_hand_side and right_hand_side.get("value") == "0":
                    parent_node = node.get("src", {})
                    youso2.append(function_name)
                    youso2.append(parent_node)
                    find_sub_value_in_function_list.append(youso2)
                    #print(f"'sub' found in function: {function_name}")
        for key, value in node.items():
            find_sub_value_in_function(value, function_name)
    elif isinstance(node, list):
        for item in node:
            find_sub_value_in_function(item, function_name)

for solidity_file in solidity_files:
    file_path = os.path.join(contract_directory, solidity_file)
    find_call_value_in_function_list = [] 
    find_sub_value_in_function_list = []
    with open(file_path, "rb") as file:
        file_data = file.read()
        try:
            ast_data = file_data.decode("utf-8")
            ast_data = json.loads(ast_data)
            if ast_data:
                find_call_value_in_function(ast_data, function_name=None)
                find_sub_value_in_function(ast_data, function_name=None)
                for a in find_call_value_in_function_list:
                    for b in find_sub_value_in_function_list:
                        if a[0]==b[0]:
                            if a[1] < b[1]:
                                res_sub += 1
                                solidity_filesY2.append(solidity_file)
                            print(f"Found substruct after 'call.value': {solidity_file}")
        except UnicodeDecodeError:
            r1 += 1
print(f"result:{res_sub}")
print(f"files: {solidity_filesY2}")

Found substruct after 'call.value': ast_output_26250.sol.json
result:1
files: ['ast_output_26250.sol.json']


どっちも100%


３. 関数修飾子

In [35]:
#できた
#call.valueが含まれる関数名
import os
import json

contract_directory = "/Users/iniad/Downloads/dataset_preprocessing_for_vulnerabilities/testcode/reentrancy"
solidity_files = os.listdir(contract_directory)
solidity_filesY3 = []
res_reentrancy = 0
def find_call_value_in_function(node, function_name):
    if isinstance(node, dict):
        if "nodeType" in node and node["nodeType"] == "FunctionDefinition":
            function_name = node.get("name")
        if "nodeType" in node and node["nodeType"] == "MemberAccess" and node.get("memberName") == "value":
            if "expression" in node and node["expression"].get("memberName") == "call":
                youso1 = []
                parent_node = node.get("src", {})
                youso1.append(function_name)
                youso1.append(parent_node)
                find_call_value_in_function_list.append(youso1)
                #print(f"'call.value' found in function: {function_name} {parent_node}")
        for key, value in node.items():
            find_call_value_in_function(value, function_name)
    elif isinstance(node, list):
        for item in node:
            find_call_value_in_function(item, function_name)

def find_sub_value_in_function(node, function_name):
    if isinstance(node, dict):
        if "nodeType" in node and node["nodeType"] == "FunctionDefinition":
            function_name = node.get("name")
        if "nodeType" in node and (node["nodeType"] == "BinaryOperation" or node.get("nodeType") == "Assignment"):
            youso2 = []
            if (node["operator"] == "-" or node["operator"] == "-="):
                parent_node = node.get("src", {})
                youso2.append(function_name)
                youso2.append(parent_node)
                find_sub_value_in_function_list.append(youso2)
                #print(f"'sub' found in function: {function_name}")
            elif node.get("operator") == "=":
                right_hand_side = node.get("rightHandSide")
                if right_hand_side and right_hand_side.get("value") == "0":
                    parent_node = node.get("src", {})
                    youso2.append(function_name)
                    youso2.append(parent_node)
                    find_sub_value_in_function_list.append(youso2)
                    #print(f"'sub' found in function: {function_name}")
        for key, value in node.items():
            find_sub_value_in_function(value, function_name)
    elif isinstance(node, list):
        for item in node:
            find_sub_value_in_function(item, function_name)
            
def find_modifier_noReentrancy_in_function(node, function_name):
    modifier_name1 = "noReentrancy"
    if isinstance(node, dict):
        if "nodeType" in node and node["nodeType"] == "FunctionDefinition":
            function_name = node.get("name")
        if "nodeType" in node and node["nodeType"] == "FunctionDefinition":
            youso3 = []
            function_name = node.get("name")
            modifiers = node.get("modifiers")
            if modifiers:
                for modifier in modifiers:
                    modifier_name = modifier["modifierName"]["name"]
                    if modifier_name == modifier_name1:
                        parent_node = node.get("src", {})
                        youso3.append(function_name)
                        youso3.append(parent_node)
                        find_modifier_noReentrancy_in_function_list.append(youso3)
                        #print(f"'noreentrancy' found in function: {function_name}")
        for key, value in node.items():
            find_modifier_noReentrancy_in_function(value, function_name)
    elif isinstance(node, list):
        for item in node:
            find_modifier_noReentrancy_in_function(item, function_name)
r1 = 0

for solidity_file in solidity_files:
    file_path = os.path.join(contract_directory, solidity_file)
    find_call_value_in_function_list = [] 
    find_sub_value_in_function_list = []
    find_modifier_noReentrancy_in_function_list = []
    with open(file_path, "rb") as file:
        file_data = file.read()
        try:
            ast_data = file_data.decode("utf-8")
            ast_data = json.loads(ast_data)
            if ast_data:
                find_call_value_in_function(ast_data, function_name=None)
                find_sub_value_in_function(ast_data, function_name=None)
                find_modifier_noReentrancy_in_function(ast_data, function_name=None)
                for a in find_call_value_in_function_list:
                    for b in find_sub_value_in_function_list:
                        if a[0]==b[0]:
                            if a[1] < b[1]:
                                if find_modifier_noReentrancy_in_function_list:
                                    for c in find_modifier_noReentrancy_in_function_list:
                                        if b[0] != c[0]:
                                            solidity_filesY3.append(solidity_file)
                                            res_reentrancy += 1
                                            print(f"Found reentrancy: {solidity_file}")
                                if not find_modifier_noReentrancy_in_function_list:
                                            solidity_filesY3.append(solidity_file)
                                            res_reentrancy += 1
                                            print(f"Found reentrancy: {solidity_file}")
        except UnicodeDecodeError:
            r1 += 1
print(f"result:{res_reentrancy}")
print(f"files: {solidity_filesY3}")

Found reentrancy: ast_output_40754.sol.json
Found reentrancy: ast_output_50013.sol.json
Found reentrancy: ast_output_40780.sol.json
Found reentrancy: ast_output_50003.sol.json
Found reentrancy: ast_output_40746.sol.json
result:5
files: ['ast_output_40754.sol.json', 'ast_output_50013.sol.json', 'ast_output_40780.sol.json', 'ast_output_50003.sol.json', 'ast_output_40746.sol.json']


In [34]:
#できた
#call.valueが含まれる関数名
import os
import json

contract_directory = "/Users/iniad/Downloads/dataset_preprocessing_for_vulnerabilities/testcode/real_ast"
solidity_files = os.listdir(contract_directory)
solidity_filesY3 = []
res_reentrancy = 0
def find_call_value_in_function(node, function_name):
    if isinstance(node, dict):
        if "nodeType" in node and node["nodeType"] == "FunctionDefinition":
            function_name = node.get("name")
        if "nodeType" in node and node["nodeType"] == "MemberAccess" and node.get("memberName") == "value":
            if "expression" in node and node["expression"].get("memberName") == "call":
                youso1 = []
                parent_node = node.get("src", {})
                youso1.append(function_name)
                youso1.append(parent_node)
                find_call_value_in_function_list.append(youso1)
                #print(f"'call.value' found in function: {function_name} {parent_node}")
        for key, value in node.items():
            find_call_value_in_function(value, function_name)
    elif isinstance(node, list):
        for item in node:
            find_call_value_in_function(item, function_name)

def find_sub_value_in_function(node, function_name):
    if isinstance(node, dict):
        if "nodeType" in node and node["nodeType"] == "FunctionDefinition":
            function_name = node.get("name")
        if "nodeType" in node and (node["nodeType"] == "BinaryOperation" or node.get("nodeType") == "Assignment"):
            youso2 = []
            if (node["operator"] == "-" or node["operator"] == "-="):
                parent_node = node.get("src", {})
                youso2.append(function_name)
                youso2.append(parent_node)
                find_sub_value_in_function_list.append(youso2)
                #print(f"'sub' found in function: {function_name}")
            elif node.get("operator") == "=":
                right_hand_side = node.get("rightHandSide")
                if right_hand_side and right_hand_side.get("value") == "0":
                    parent_node = node.get("src", {})
                    youso2.append(function_name)
                    youso2.append(parent_node)
                    find_sub_value_in_function_list.append(youso2)
                    #print(f"'sub' found in function: {function_name}")
        for key, value in node.items():
            find_sub_value_in_function(value, function_name)
    elif isinstance(node, list):
        for item in node:
            find_sub_value_in_function(item, function_name)
            
def find_modifier_noReentrancy_in_function(node, function_name):
    modifier_name1 = "noReentrancy"
    if isinstance(node, dict):
        if "nodeType" in node and node["nodeType"] == "FunctionDefinition":
            function_name = node.get("name")
        if "nodeType" in node and node["nodeType"] == "FunctionDefinition":
            youso3 = []
            function_name = node.get("name")
            modifiers = node.get("modifiers")
            if modifiers:
                for modifier in modifiers:
                    modifier_name = modifier["modifierName"]["name"]
                    if modifier_name == modifier_name1:
                        parent_node = node.get("src", {})
                        youso3.append(function_name)
                        youso3.append(parent_node)
                        find_modifier_noReentrancy_in_function_list.append(youso3)
                        #print(f"'noreentrancy' found in function: {function_name}")
        for key, value in node.items():
            find_modifier_noReentrancy_in_function(value, function_name)
    elif isinstance(node, list):
        for item in node:
            find_modifier_noReentrancy_in_function(item, function_name)
r1 = 0

for solidity_file in solidity_files:
    file_path = os.path.join(contract_directory, solidity_file)
    find_call_value_in_function_list = [] 
    find_sub_value_in_function_list = []
    find_modifier_noReentrancy_in_function_list = []
    with open(file_path, "rb") as file:
        file_data = file.read()
        try:
            ast_data = file_data.decode("utf-8")
            ast_data = json.loads(ast_data)
            if ast_data:
                find_call_value_in_function(ast_data, function_name=None)
                find_sub_value_in_function(ast_data, function_name=None)
                find_modifier_noReentrancy_in_function(ast_data, function_name=None)
                for a in find_call_value_in_function_list:
                    for b in find_sub_value_in_function_list:
                        if a[0]==b[0]:
                            if a[1] < b[1]:
                                if find_modifier_noReentrancy_in_function_list:
                                    for c in find_modifier_noReentrancy_in_function_list:
                                        if b[0] != c[0]:
                                            solidity_filesY3.append(solidity_file)
                                            res_reentrancy += 1
                                            print(f"Found reentrancy: {solidity_file}")
                                if not find_modifier_noReentrancy_in_function_list:
                                            solidity_filesY3.append(solidity_file)
                                            res_reentrancy += 1
                                            print(f"Found reentrancy: {solidity_file}")
        except UnicodeDecodeError:
            r1 += 1
print(f"result:{res_reentrancy}")
print(f"files: {solidity_filesY3}")

Found reentrancy: ast_output_26250.sol.json
result:1
files: ['ast_output_26250.sol.json']


どちらも100%

全体の結果

データセットXもデータセットYも全て正確に調べられた